In [1]:
import os
import networkx
import pickle
from namespace.graph.json_processor import get_data_matrix
from namespace.graph.comp_printer import print_comp

In [2]:
with open('./poker_strategy/graph.pickle', 'rb') as f:
    G = pickle.load(f)
champion_matrix, trait_matrix, _ = get_data_matrix()

The graph `G` is a `networkx.DiGraph` object. Each of its edge points from a lower level comp to a higher level comp. We can both work from high level to low level or low level to high level. Since high level comps are more important for players, let's make a reverse of the graph.

In [3]:
G_rev = G.reverse()
levels = [5, 6, 7, 8, 9]
subgraph_path = './subgraphs/'
roots = [node[0] for node in G_rev.nodes(data=True) if node[1]['layer'] == len(levels) - 1]
if not os.path.exists(subgraph_path):
    os.mkdir(subgraph_path)
for index, root in enumerate(roots):
    cur_G = networkx.DiGraph()
    champ_names, _, _ = print_comp(root, champion_matrix, trait_matrix)
    cur_G.add_node(str(champ_names), layer=len(levels) - 1)
    descendants = networkx.algorithms.dag.descendants(G_rev, root)
    for bit_code in descendants:
        champ_names, _, _ = print_comp(bit_code, champion_matrix, trait_matrix)
        cur_G.add_node(str(champ_names), layer=levels.index(len(champ_names)))
    for i in cur_G.nodes(data=True):
        for j in cur_G.nodes(data=True):
            if i[1]['layer'] - j[1]['layer'] == 1:
                cur_G.add_edge(i[0], j[0])
    networkx.write_graphml(cur_G, subgraph_path + f'tansition_{index}.graphml')

we save each subgraph to the folder `./subgraphs/` in `.graphml` format, each subgraph represent a transition tree, in this case from level 5 to level 9. Use Cytoscape to visualize it.